In [ ]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score

In [ ]:
tsel = pd.read_csv(r'Prepro_Pelayanan.csv', usecols=['Username','Text','Pelayanan','text_result'])
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [ ]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [ ]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,admin sinyal telkomsel rumah daerah rawabunga ...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=10000)
vectors = matrix.fit_transform(tsel['text_result']).toarray()

In [ ]:
X = vectors
y = tsel['Pelayanan']


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=45) #split data disesuaikan dengan rasio (70:30, 80:10, 90:10)

## Klasifikasi NBSVM

In [ ]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

In [ ]:
x_train_csr = sparse.csr_matrix(x_train)
x_test_csr = sparse.csr_matrix(x_test)
model = NBSVM()

model.fit(x_train_csr,y_train)
y_pred = model.predict(x_test_csr)
y_true = y_test

### random 15

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 352  277   55]
 [ 209 3673  150]
 [  40  195  147]]
              precision    recall  f1-score   support

        -1.0       0.59      0.51      0.55       684
         0.0       0.89      0.91      0.90      4032
         1.0       0.42      0.38      0.40       382

    accuracy                           0.82      5098
   macro avg       0.63      0.60      0.62      5098
weighted avg       0.81      0.82      0.81      5098

Precision:  0.6298106723560305
Recall:  0.6034663128516463
F1:  0.6155927893052783
Accuracy:  0.8183601412318556


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.706229867541737

### random 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 327  256   71]
 [ 255 3669  181]
 [  55  152  132]]
              precision    recall  f1-score   support

        -1.0       0.51      0.50      0.51       654
         0.0       0.90      0.89      0.90      4105
         1.0       0.34      0.39      0.37       339

    accuracy                           0.81      5098
   macro avg       0.59      0.59      0.59      5098
weighted avg       0.81      0.81      0.81      5098

Precision:  0.5856734051802642
Recall:  0.5943895314369482
F1:  0.589525336110726
Accuracy:  0.8097293056100432


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7141769805162111

### random 25

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 353  246   68]
 [ 266 3641  144]
 [  43  174  163]]
              precision    recall  f1-score   support

        -1.0       0.53      0.53      0.53       667
         0.0       0.90      0.90      0.90      4051
         1.0       0.43      0.43      0.43       380

    accuracy                           0.82      5098
   macro avg       0.62      0.62      0.62      5098
weighted avg       0.81      0.82      0.82      5098

Precision:  0.6214921642666688
Recall:  0.6189910576159646
F1:  0.6202323371031089
Accuracy:  0.8154178109062378


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7235261325996238

### Random 35

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 359  271   53]
 [ 252 3637  177]
 [  51  156  142]]
              precision    recall  f1-score   support

        -1.0       0.54      0.53      0.53       683
         0.0       0.89      0.89      0.89      4066
         1.0       0.38      0.41      0.39       349

    accuracy                           0.81      5098
   macro avg       0.61      0.61      0.61      5098
weighted avg       0.81      0.81      0.81      5098

Precision:  0.6063158683257615
Recall:  0.6089966485789765
F1:  0.6074791027210501
Accuracy:  0.811690859160455


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7175233576067411

### Random 45

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 336  253   64]
 [ 261 3655  174]
 [  42  173  140]]
              precision    recall  f1-score   support

        -1.0       0.53      0.51      0.52       653
         0.0       0.90      0.89      0.89      4090
         1.0       0.37      0.39      0.38       355

    accuracy                           0.81      5098
   macro avg       0.60      0.60      0.60      5098
weighted avg       0.81      0.81      0.81      5098

Precision:  0.5972685955855414
Recall:  0.6008524892884454
F1:  0.5989143313550643
Accuracy:  0.8103177716751667


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7105180873127678